## Arboles de decisión: Hiperparámetros, Random Forest y Optimización de Parámetros

In [1]:
import os
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

In [2]:
df = pd.read_csv(os.path.join('../Datasets/diabetes.csv'))
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
feature_cols = ['Pregnancies', 'Insulin', 'BMI', 'Age','Glucose','BloodPressure','DiabetesPedigreeFunction']
X = df[feature_cols]
Y = df["Outcome"]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1) # 70% training, 30% test
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(537, 7) (231, 7) (537,) (231,)


In [4]:
# baseline no incluye poda (max_depth)
treev1 = DecisionTreeClassifier()
treev1.fit(X_train, Y_train)

DecisionTreeClassifier()

In [5]:
Y_pred = treev1.predict(X_test)
Y_pred

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0])

In [6]:
def metricas_desempenio(tree):
    print('accuracy del clasificador - version 1 : {0:.2f}'.format(accuracy_score(Y_test, tree.predict(X_test))))
    print('matriz de confusión del clasificador - version 1: \n {0}'.format(confusion_matrix(Y_test, tree.predict(X_test))))
    print('precision del clasificador - version 1 : {0:.2f}'.format(precision_score(Y_test, tree.predict(X_test))))
    print('recall del clasificador - version 1 : {0:.2f}'.format(recall_score(Y_test, tree.predict(X_test))))
    print('f1 del clasificador - version 1 : {0:.2f}'.format(f1_score(Y_test, tree.predict(X_test))))
metricas_desempenio(treev1)

accuracy del clasificador - version 1 : 0.68
matriz de confusión del clasificador - version 1: 
 [[113  33]
 [ 42  43]]
precision del clasificador - version 1 : 0.57
recall del clasificador - version 1 : 0.51
f1 del clasificador - version 1 : 0.53


In [7]:
#Ajustar algunos hiperparámetros
tree_v2 = DecisionTreeClassifier(criterion="entropy", max_depth=3)
tree_v2.fit(X_train, Y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=3)

In [8]:
metricas_desempenio(tree_v2)

accuracy del clasificador - version 1 : 0.77
matriz de confusión del clasificador - version 1: 
 [[124  22]
 [ 31  54]]
precision del clasificador - version 1 : 0.71
recall del clasificador - version 1 : 0.64
f1 del clasificador - version 1 : 0.67


### Hiperparámetros para ajustar la complejidad del modelo

__[DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)__
* class_weight=None importancia relativa de los valores de clasificación
* criterion='entropy'/'gini'
* max_depth=3 distancia max entre a raiz y las hojas
* max_features=None numero max de variables a considerar
* max_leaf_nodes=20 numero max de hojas
* min_impurity_decrease=0.0
* min_impurity_split=None (Deprecado)
* min_samples_leaf=1 Podar si quedan menos que este numero de ejemplos 
* min_samples_split=2 Continuar si quedan al menos esta cantidad de ejemplos
* min_weight_fraction_leaf=0.0 Porcentaje minimo de ejemplo para continuar


<img src="./img/28-complejidad-vs-accuracy.png" style="width:600px"/>

**Más allá de cierto umbral, la complejidad del modelo afecta negativamente el desempeño debido al sobreajuste**

## Sobreajuste

__[Sobreajuste](https://es.wikipedia.org/wiki/Sobreajuste)__

<img src="./img/29-overfitting-sobreajuste.png" style="width:600px"/>

<img src="./img/30.0-underfitting.png" style="width:600px"/>

<img src="./img/30.1-origen-del-sobreajuste.png" style="width:600px"/>

**Como evitar?**
En el caso particular de los árboles de decisión, reducir nodos del arbol cuando no incrementan los indicadores con una buena cantidad de datos de prueba (**poda - pruning**)

### Ensemble learning

__[Ensemble learning](https://en.wikipedia.org/wiki/Ensemble_learning)__

<img src="./img/31-ensemble-learning.png" style="width:600px"/>

<img src="./img/32-ensemble-arboles.png" style="width:600px"/>

<img src="./img/33-ensemble-arboles.png" style="width:600px"/>


## Random Forest

__[RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)__

Se crean varios árboles **INDEPENDIENTES** variando los casos/observaciones del conjunto de entrenamiento y/o las variables empleadas durante el proceso de entrenamiento.

Las predicciones de cada modelo (árbol) tienen el mismo peso y el resultado final es el voto de mayoría

**Parámetros:**
* **n_estimators** número de clasificadores, árboles en este caso.

Los valores adecuados para este y otros parámetros se obtienen via experimentación (prueba y error). Si es posible, se recomienda tener varios conjuntos de prueba para seleccionar el modelo con el mejor desempeño (promedio) en todos los conjunto de prueba

In [9]:
from sklearn.ensemble import RandomForestClassifier
#Ajustar n_estimators puede reducir la posiblidad de overfitting
tree_v3 = RandomForestClassifier(n_estimators=10)
tree_v3

RandomForestClassifier(n_estimators=10)

In [10]:
tree_v3.fit(X_train, Y_train)

RandomForestClassifier(n_estimators=10)

In [11]:
metricas_desempenio(tree_v3)

accuracy del clasificador - version 1 : 0.77
matriz de confusión del clasificador - version 1: 
 [[127  19]
 [ 34  51]]
precision del clasificador - version 1 : 0.73
recall del clasificador - version 1 : 0.60
f1 del clasificador - version 1 : 0.66


## Gradient Boosted Trees
Los arboles se construyen en **secuencia** a partir de una fracción del conjunto de entrenamiento; la idea central es que el siguiente árbol corriga los errores del anterior.:

Inicialmente, todos los ejemplos tienen la misma probabilidad de ser seleccionados. A partir del segundo árbol, los ejemplos que fueros incorrectamente clasificados por el árbol anterior tienen mayor probabilidad de ser seleccionados. (para detectar patrones que no fueron detectados por el anterior)

En consecuencia, cada árbol se crea a partir de una fracción diferente del conjunto de entrenamiento. En la colección final, la clasificación de cada árbol tiene un peso mayor en función del desempeño obtenido con el conjuto de entrenamiento.

__[py-xgboost](https://anaconda.org/anaconda/py-xgboost)__

__[xgboost](https://xgboost.readthedocs.io/en/latest/python/python_api.html)__


In [12]:
from xgboost.sklearn import XGBClassifier

/usr/local/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [13]:
tree_v4 = XGBClassifier(n_estimators=10)
tree_v4

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=10, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None)

In [14]:
tree_v4.fit(X_train, Y_train)

[17:26:53] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=10, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [15]:
metricas_desempenio(tree_v4)

accuracy del clasificador - version 1 : 0.80
matriz de confusión del clasificador - version 1: 
 [[131  15]
 [ 32  53]]


precision del clasificador - version 1 : 0.78
recall del clasificador - version 1 : 0.62
f1 del clasificador - version 1 : 0.69


/usr/local/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/usr/local/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/usr/local/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/usr/local/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pa

### Parámetros que se pueden emplear para evitar sobre ajuste (overfitting) 
* **n_estimators**: a mayor cantidad de ejemplos, se puede incrementar el valor n_estimators para evitar sobreajuste

* **learning_rate**, determina la probabilidad de que un ejemplo sea seleccionado en la siguiente iteracion, se recomienda un valor entre 0.1 - 0.2 para reducir la probabilidad de que se produzca overfitting

* **subsample**, permite controlar el tamaño de la fracción del conjunto de entrenamiento  para cada iteración. Mientras más bajo el valor, más probabilidad hay de que los conjuntos de entrenamiento entre iteraciones sean diferentes (a mayor diferencia, menos probabilidad de que se produzca overfitting). Se recomienda valores entre 0.5 - 1.0

* **colsample_bytree**, permite controlar la fracción de las variables empleadas para entrenar los árboles en cada iteración. Se recomienda valores entre 0.5 - 1.0

## Optimización de parámetros

Objetivo: encontrar la mejor combinación de hiper-parámetros para obtener el clasificador con el mejor desempeño.

Para evitar probar manualmente todas las posibles combinaciones de valores para todos los posibles parámetros que resultan en un buen desempeño, se emplean técnicas de optimización para evitar buscar en todo el espacio de posible valores y garantizar al mismo tiempo un buen desempeño del clasificador. 


__[hyperopt (Distributed Hyperparameter Optimization)](https://github.com/hyperopt/hyperopt)__ es el módulo python que facilita realizar esta tarea.

In [16]:
#conda install -c conda-forge hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK,Trials

In [17]:
space = {
    'x':hp.quniform('x',-1,1,1), #probar con valores entre -10 - 10, con incrementos de 1 
}

In [18]:
def objective(params):
    x = int(params['x'])
    return {'loss':x ** 2,'status':STATUS_OK}   

In [19]:
trials = Trials()

In [20]:
best = fmin(objective, space, algo=tpe.suggest, trials=trials, max_evals=5)
print(best)

  0%|                                                                                                                    | 0/5 [00:00<?, ?trial/s, best loss=?]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 668.44trial/s, best loss: 0.0]


{'x': -0.0}


In [21]:
#Probar valores entre 100 - 1000, con incrementos de 1 - con igual probabilidad de ser seleccionado: 
#'n_estimators':hp.quniform('n_estimators',100,1000,1)
#Crear un diccionario que contiene la configuración para generar diferentes valores para cada parámetro; en este ejemplo,
#para el algoritmo XGBClassifier.
space =  {
    'n_estimators':hp.quniform('n_estimators',100,1000,1), #probar con valores entre 100 - 100, con incrementos de 1 
    'learning_rate':hp.quniform('learning_rate',0.025,0.5,0.025),
    'max_depth':hp.quniform('max_depth',1,13,1),
    'subsample': hp.quniform('subsample',0.5,1,0.05),
    'colsample_bytree':hp.quniform('colsample_bytree',0.5,1,0.05),
    'nthread':6, #cuando se posible, paralelizar el procesamiento empleando hasta 6 hilos
    'silent':1 #si ocurre un error, continuar con la ejecución
}

In [22]:
#Es necesario definir una función de manera tal que cuando alcance el valor mínimo, esto implique que el clasificador
#ha alanzado en mejor desempeño. 
#En el ejemplo siguiente, el menor valor posible para esta función (0), si se da cuando accuracy = 1.
def objective(params):
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])  
    clf = XGBClassifier(**params) #https://treyhunner.com/2018/10/asterisks-in-python-what-they-are-and-how-to-use-them/
    clf.fit(X_train, Y_train)   
    accuracy = accuracy_score(Y_test, clf.predict(X_test))
    return {'loss': 1 - accuracy, 'status': STATUS_OK}

In [23]:
#https://github.com/hyperopt/hyperopt/wiki/FMin#12-attaching-extra-information-via-the-trials-object
#fmin Itera 100 veces y retorna la combinación de parámetros que generan el menor valor para la función 'objective'
trials = Trials()
best = fmin(objective,space,algo=tpe.suggest,trials=trials,max_evals=100)
print(best)

  0%|                                                                                                                  | 0/100 [00:00<?, ?trial/s, best loss=?]

[17:26:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


  0%|                                                                                                                  | 0/100 [00:00<?, ?trial/s, best loss=?]

[17:26:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
  0%|                                                                                                                  | 0/100 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

/usr/local/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):



  1%|▉                                                                                       | 1/100 [00:00<00:28,  3.50trial/s, best loss: 0.2640692640692641]

...

[17:27:20] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 99%|█████████████████████████████████████████████████████████████████████████████████████▏| 99/100 [00:25<00:00,  3.76trial/s, best loss: 0.19480519480519476]

/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

/usr/local/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):

/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructi

100%|█████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:26<00:00,  4.23trial/s, best loss: 0.19480519480519476]

100%|█████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:26<00:00,  3.84trial/s, best loss: 0.19480519480519476]


{'colsample_bytree': 0.9500000000000001, 'learning_rate': 0.07500000000000001, 'max_depth': 1.0, 'n_estimators': 455.0, 'subsample': 0.7000000000000001}


In [24]:
best['n_estimators'] = int(best['n_estimators'])
best['max_depth'] = int(best['max_depth'])                      

In [25]:
tree_v5 = XGBClassifier(**best)
tree_v5

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.9500000000000001,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.07500000000000001, max_delta_step=None,
              max_depth=1, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=455, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None,
              reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
              subsample=0.7000000000000001, tree_method=None,
              validate_parameters=None, verbosity=None)

In [26]:
tree_v5.fit(X_train, Y_train)

[17:27:20] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9500000000000001,
              enable_categorical=False, gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.07500000000000001, max_delta_step=0, max_depth=1,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=455, n_jobs=8, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=0.7000000000000001, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [27]:
metricas_desempenio(tree_v5)

accuracy del clasificador - version 1 : 0.81
matriz de confusión del clasificador - version 1: 
 [[132  14]
 [ 31  54]]
precision del clasificador - version 1 : 0.79
recall del clasificador - version 1 : 0.64
f1 del clasificador - version 1 : 0.71


/usr/local/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/usr/local/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/usr/local/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/usr/local/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pa